In [168]:
import os
import scipy
import math
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import norm
from tqdm import tqdm

# Define helper function that calculates OR,LR, and their confidence intervals
OR CI formula: https://www.ncbi.nlm.nih.gov/books/NBK431098/

LR CI formula: https://stats.stackexchange.com/questions/61349/how-to-calculate-the-confidence-intervals-for-likelihood-ratios-from-a-2x2-table


In [169]:
def calc_or(t11,t12,t21,t22,sig_level=0.95):
    odds_ratio, pvalue = stats.fisher_exact([[t11, t12], [t21, t22]])
    alpha = 1-sig_level
    # OR 95% CI
    if t11!=0 and t12!=0 and t21!=0 and t22!=0:
        standard_error = np.sqrt((1/t11)+(1/t12)+(1/t21)+(1/t22))
    else:
        standard_error = None
    if odds_ratio == odds_ratio and standard_error != None:
        ci_lower = np.exp(np.log(odds_ratio) - norm.ppf(1-(alpha/2))*standard_error)
        ci_upper = np.exp(np.log(odds_ratio) + norm.ppf(1-(alpha/2))*standard_error)
    else:
        ci_lower = np.NaN
        ci_upper = np.NaN
    
    return odds_ratio,ci_lower,ci_upper,pvalue

def calc_lr_plus(t11,t12,t21,t22,sig_level=0.95):
    alpha = 1-sig_level
    a,b,c,d = t11,t12,t21,t22
    specificity = d/(b+d)
    sensitivity = a/(a+c)
    
    if (1-specificity) == 0:
        return np.Inf,np.NaN,np.NaN
    
    lr_pos = sensitivity / (1-specificity)
    
    
    if a!=0 and b!=0:
        sigma2 = (1/a) - (1/(a+c)) + (1/b) - (1/(b+d))
        lower_pos = lr_pos * np.exp(-norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
        upper_pos = lr_pos * np.exp(norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
    
    elif a==0 and b==0:
        lower_pos = 0
        upper_pos = np.Inf
    
    elif a==0 and b!=0:
        a_temp = 1/2
        specificity_temp = d/(b+d)
        sensitivity_temp = a_temp/(a+c)
        lr_pos_temp = sensitivity_temp/(1-specificity_temp)
        lower_pos = 0
        sigma2 = (1/a_temp) - (1/(a_temp+c)) + (1/b) - (1/(b+d))
        upper_pos = lr_pos_temp * np.exp(norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
    
    elif a!=0 and b == 0:
        b_temp = 1/2
        specificity_temp = d/(b_temp+d)
        sensitivity_temp = a/(a+c)
        lr_pos_temp = sensitivity/(1-specificity_temp)
        sigma2 = (1/a) - (1/(a+c)) + (1/b_temp) - (1/(b_temp+d))
        lower_pos = lr_pos_temp*np.exp(-norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
        upper_pos = np.Inf
    
    elif (a==(a+c)) and (b==(b+d)):
        a_temp = a-(1/2)
        b_temp = b-(1/2)
        
        specificity_temp = d/(b_temp+d)
        sensitivity_temp = a_temp/(a+c)
        
        lr_pos_temp = sensitivity_temp/(1-specificity_temp)
        
        sigma2 = (1/a_temp) - (1/(a_temp+c)) + (1/b_temp) - (1/(b_temp+d))
        
        lower_pos = lr_pos_temp * np.exp(-norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
        upper_pos = lr_pos>temp * np.exp(norm.ppf(1-(alpha/2))*np.sqrt(sigma2))
        
    return lr_pos,lower_pos,upper_pos

# Define Helper functions that parse text outputs from WEKA

In [170]:
def parse_weka_apriori_txt_file(txt_path,class_val=1):
    to_return = []
    txt_file = open(txt_path, "r")
    rules_flag = False
    for line in txt_file:
        if rules_flag and line != "\n":
            parsed_dict = parse_weka_aprior_line(line)
            # Rules which class_val is not 1 are not considered
            if parsed_dict['class']['class_val'] == class_val:
                to_return.append(parsed_dict)
        elif "Best rules found" in line:
            rules_flag = True
    return to_return

def parse_weka_aprior_line(line_text):
    line_text = line_text.strip()
    rule_class_split = line_text.split("==>")
    rule_text = rule_class_split[0].strip().split(" ")
    rule_id = rule_text[0].replace(".","")
    rule_pairs = [rule_text[i] for i in range(1,len(rule_text) - 1)]
    kv_pairs = {pair.split("=")[0]:int(float(pair.split("=")[1])) for pair in rule_pairs}
    rule_total = int(rule_text[len(rule_text) - 1])
    
    class_text = rule_class_split[1].strip().split(" ")
    class_text_vals = class_text[0].split("=")
    class_name = class_text_vals[0]
#     print(class_text_vals[1])
    class_val = int(float(class_text_vals[1]))
    class_count = int(class_text[1])
    
    rule_dict = {
        "id": rule_id,
        "rules":{
        "rule_pairs": kv_pairs,
        "rule_total": rule_total
        },
        "class":{
            "class_name":class_name,
            "class_val": class_val,
            "class_count":class_count
        },
        "conf": class_count/rule_total
    }
        
    return rule_dict

def convert_parsed_weka_to_vec(parsed_obj,all_attributes,class_val_total,non_class_val_total):
    to_return_vec = []
    
    # id
    to_return_vec.append(parsed_obj["id"])
        
    # item size
    obj_rule_pairs = parsed_obj["rules"]["rule_pairs"]
    to_return_vec.append(len(obj_rule_pairs))
    
    # add rules
    for rule in all_attributes:
        if rule in obj_rule_pairs:
            to_return_vec.append(obj_rule_pairs[rule])
        else:
            to_return_vec.append(None)

    # class val
    class_val = int(parsed_obj["class"]["class_val"])
    to_return_vec.append(class_val)
    
    # coverage_raw_count
    rule_total = int(parsed_obj["rules"]["rule_total"])
    to_return_vec.append(rule_total)
    
    # support_raw_count
    class_count = int(parsed_obj["class"]["class_count"])
    to_return_vec.append(class_count)
    
    # coverage
    to_return_vec.append(rule_total/(class_val_total+non_class_val_total))
    
    # support
    to_return_vec.append(class_count/(class_val_total+non_class_val_total))
    
    # posterior/confidence
    to_return_vec.append(class_count/rule_total)
    
    # Odds Ratio and p value
    if class_val == 1:
        t11 = class_count
        t12 = rule_total - t11
        t21 = class_val_total - t11
        t22 = non_class_val_total - t12
    # When class_val = 0 (protective rules), GDM positive patients become control and negative patients become cases
    elif class_val == 0:
        t11 = class_count
        t12 = rule_total - t11
        t21 = non_class_val_total - t11
        t22 = class_val_total - t12
    
    if t11<0 or t12<0 or t21<0 or t22<0:
        print(f"t11:{t11},t12:{t12},t21:{t21},t22:{t22}")
        print(parsed_obj)

    # Calculates OR, CI, and p-value from fisher test
    odds_ratio,or_lower_ci,or_upper_ci,p_value = calc_or(t11,t12,t21,t22)

    to_return_vec.append(odds_ratio)
    to_return_vec.append(or_lower_ci)
    to_return_vec.append(or_upper_ci)
    to_return_vec.append(p_value)
    
    # Calculates LR+, CI
    lr_plus,lr_lower_ci,lr_upper_ci = calc_lr_plus(t11,t12,t21,t22)
    
    to_return_vec.append(lr_plus)
    to_return_vec.append(lr_lower_ci)
    to_return_vec.append(lr_upper_ci)
    
    return(to_return_vec)
    
def convert_weka_aprior_doc_to_df(txt_path,all_attributes,class_val_total,non_class_val_total):
    parsed_rule_obj_list = parse_weka_apriori_txt_file(txt_path)
    print(f"All Rules are now parsed into rule dictionary. Total rules: {len(parsed_rule_obj_list)}")
    print("Converting rule dictionary into vectors...")
    all_rules_vecs = []
    for idx in tqdm(range(len(parsed_rule_obj_list))):
        cur_obj = parsed_rule_obj_list[idx]
        cur_vec = convert_parsed_weka_to_vec(cur_obj,all_attributes,class_val_total,non_class_val_total)
        all_rules_vecs.append(cur_vec)
    
#     all_rules_vecs = [convert_parsed_weka_to_vec(obj,all_attributes,class_val_total,non_class_val_total) for obj in parsed_rule_obj_list]
    
    column_headers = ["rule_id","num_clauses"]
    column_headers += all_attributes
    
    column_headers_part_2 = ["class_val","coverage_count","support_count",
                             "coverage","support","confidence/posterior","odds_ratio","OR_95CI_lower","OR_95CI_upper","p_value",
                             "LR+","LR+_95CI_lower","LR+_95CI_upper"]
    
    column_headers += column_headers_part_2
    
    big_df = pd.DataFrame(columns=column_headers,data=all_rules_vecs)
    return big_df

# Loads the necessary files and declare variables
Loads the original data file (to find the number of cases and controls)

Loads the text file containing the rules

Set the columns that will be extrated (clause names in rules)

In [208]:
# Replace source_df path with your path to the processed data produced by the Preprocessing notebook
source_df = pd.read_csv("data_source/baseline_data_2021_10_13_AR_Ready.csv")

# Replace txt_path with your path to the WEKA output log. See README on how to produce the log
txt_path = "weka_output/weka_AR_2020_10_13.txt"


case_total = len(source_df[source_df['GDM'] == 1])
control_total = len(source_df[source_df['GDM'] == 0])
print(f"Total Cases: {case_total}")
print(f"Total Controls: {control_total}")
print(f"Total Participants: {case_total + control_total}")

all_attributes = ["AgeCat","BMI_Cat","Race",'Diabetes_History_Cat','PCOS_Cat','High_BP_Cat','Gravidity_Cat',
                  'Exercise_Cat','Smoking_Cat','Drinking_Cat','Drug_Cat','Poverty_Cat','Education_Cat']


Total Cases: 376
Total Controls: 8775
Total Participants: 9151


# Runs the rules to dataframe conversion

In [175]:
big_rules_df = convert_weka_aprior_doc_to_df(txt_path,all_attributes,case_total,control_total)
big_rules_df

All Rules are now parsed into rule dictionary. Total rules: 132649
Converting rule dictionary into vectors...


100%|██████████| 132649/132649 [04:25<00:00, 498.84it/s]


,rule_id,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,Exercise_Cat,...,coverage,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper
0,655705,6,NaN,NaN,NaN,1.0,NaN,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
1,688458,7,NaN,NaN,NaN,1.0,1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
2,688459,7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
3,688460,7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
4,726752,8,NaN,NaN,NaN,1.0,1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132644,1606103,9,2.0,2.0,1.0,NaN,1.0,1.0,1.0,NaN,...,0.072233,0.000546,0.007564,0.166800,0.068773,0.404552,1.293965e-07,0.177879,0.074242,0.426190
132645,1606104,8,2.0,2.0,1.0,NaN,NaN,1.0,NaN,2.0,...,0.072342,0.000546,0.007553,0.166525,0.068660,0.403885,1.292404e-07,0.177609,0.074129,0.425540
132646,1606105,7,2.0,2.0,1.0,NaN,NaN,1.0,NaN,2.0,...,0.072451,0.000546,0.007541,0.166252,0.068547,0.403219,1.293303e-07,0.177339,0.074017,0.424891
132647,1606106,7,2.0,2.0,1.0,NaN,NaN,1.0,1.0,NaN,...,0.073981,0.000546,0.007386,0.162507,0.067008,0.394111,9.095415e-08,0.173644,0.072480,0.416010


## Test querying the dataframe (optional)

In [176]:
big_rules_df_filled_na = big_rules_df.copy()
big_rules_df_filled_na[clause_columns] = big_rules_df[clause_columns].fillna(-1)
big_rules_df_filled_na

,rule_id,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,Exercise_Cat,...,coverage,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper
0,655705,6,-1.0,-1.0,-1.0,1.0,-1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
1,688458,7,-1.0,-1.0,-1.0,1.0,1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
2,688459,7,-1.0,-1.0,-1.0,1.0,-1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
3,688460,7,-1.0,-1.0,-1.0,1.0,-1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
4,726752,8,-1.0,-1.0,-1.0,1.0,1.0,2.0,2.0,NaN,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132644,1606103,9,2.0,2.0,1.0,-1.0,1.0,1.0,1.0,NaN,...,0.072233,0.000546,0.007564,0.166800,0.068773,0.404552,1.293965e-07,0.177879,0.074242,0.426190
132645,1606104,8,2.0,2.0,1.0,-1.0,-1.0,1.0,-1.0,2.0,...,0.072342,0.000546,0.007553,0.166525,0.068660,0.403885,1.292404e-07,0.177609,0.074129,0.425540
132646,1606105,7,2.0,2.0,1.0,-1.0,-1.0,1.0,-1.0,2.0,...,0.072451,0.000546,0.007541,0.166252,0.068547,0.403219,1.293303e-07,0.177339,0.074017,0.424891
132647,1606106,7,2.0,2.0,1.0,-1.0,-1.0,1.0,1.0,NaN,...,0.073981,0.000546,0.007386,0.162507,0.067008,0.394111,9.095415e-08,0.173644,0.072480,0.416010


In [177]:
def clause_dict_to_query(clause_dict):
    return " and ".join([str(key) + " == " + str(clause_dict[key]) for key in clause_dict])

# The rule(s) associated with Age Category 3 and nothing else
# Two rules are returned because we did not filter for rules which has class_val == 0
receive_input = {'AgeCat': 3,
                 'BMI_Cat': -1,
                 'Race': -1,
                 'Diabetes_History_Cat': -1,
                 'PCOS_Cat': -1,
                 'High_BP_Cat': -1,
                 'Gravidity_Cat': -1,
                 'Exercise_Cat': -1,
                 'Smoking_Cat': -1,
                 'Drinking_Cat': -1,
                 'Drug_Cat': -1,
                 'Poverty_Cat': -1,
                 'Education_Cat': -1
                }
test_query = clause_dict_to_query(receive_input)
test_query

big_rules_df_filled_na.query(test_query)

,rule_id,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,Exercise_Cat,...,coverage,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper
857,1474316,3,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,...,0.002076,0.000765,0.368421,13.852981,5.422465,35.390749,0.000006,13.613697,5.390619,34.380605
860,1474319,4,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,...,0.002076,0.000765,0.368421,13.852981,5.422465,35.390749,0.000006,13.613697,5.390619,34.380605
1994,1475453,2,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,...,0.003169,0.000874,0.275862,9.062112,3.987384,20.595426,0.000015,8.890578,3.963968,19.940216
1995,1475454,3,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,...,0.003169,0.000874,0.275862,9.062112,3.987384,20.595426,0.000015,8.890578,3.963968,19.940216
2172,1475631,3,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,...,0.002076,0.000546,0.263158,8.433770,3.021806,23.538395,0.000823,8.334916,3.017915,23.019482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78946,1552405,5,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,...,0.014752,0.000656,0.044444,1.086864,0.476237,2.480429,0.825216,1.085477,0.481912,2.444974
85460,1558919,5,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,...,0.013332,0.000546,0.040984,0.997305,0.405041,2.455594,1.000000,0.997340,0.409939,2.426430
85484,1558943,6,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,...,0.013332,0.000546,0.040984,0.997305,0.405041,2.455594,1.000000,0.997340,0.409939,2.426430
88784,1562243,4,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,...,0.013878,0.000546,0.039370,0.955879,0.388507,2.351837,1.000000,0.956466,0.393436,2.325225


## Provide text intepretation for each rule


In [183]:
mapper_dict = {
    "AgeCat": {
        0: "No Data",
        1: "[0-17]",
        2: "[18-25]",
        3: "[26-34]",
        4: "[35-39]",
        5: "40+"
    },
    "BMI_Cat": {
        0: "No Data",
        1: "[0-18.5)",
        2: "[18.5-25)",
        3: "[25-30)",
        4: "[30-35)",
        5: "35+"
    },
    "Race": {
        1: "Non-Hispanic White",
        2: "Non-Hispanic Black",
        3: "Hispanic",
        4: "American Indian",
        5: "Asian",
        6: "Native Hawaiian",
        7: "Other",
        8: "Multiracial",
        9: "Missing"
    },
    "Diabetes_History_Cat": {
        0: "Missing Data",
        1: "No Family DM History",
        2: "Has Family DM History"
    },
    "PCOS_Cat": {
        0: "Missing Data",
        1: "No PCOS",
        2: "Has PCOS"
    },
    "High_BP_Cat": {
        0: "Missing",
        1: "No High Blood Pressure",
        2: "Has High Blood Pressure"
    },
    "Gravidity_Cat": {
        0: "Missing",
        1: "First time Pregnant",
        2: "Was Pregnant Previously"
    },
    'Exercise_Cat': {
        0: "Missing Data",
        1: "Low Physical Activity (METs < 450)",
        2: "High Physical Activity (METs >= 450)"
    },
    'Smoking_Cat': {
        0: "Missing Data",
        1: "No Tobacco use within 3 months",
        2: "Tobacco use within 3 months"
    },
    'Drinking_Cat': {
        0: "Missing Data",
        1: "No Alcohol consumption within 3 months",
        2: "Consumed Alcohol within 3 months"
    },
    'Drug_Cat': {
        0: "Missing Data",
        1: "No Drug use within 1 month",
        2: "Used drug within 1 month"
    },
    'Poverty_Cat': {
        0: "Missing Data",
        1: "Below Federal poverty Level",
        2: "Above federal poverty level"
    },
    'Education_Cat': {
        0: "Missing Data",
        1: "No College Degree",
        2: "Obtained College Degree"
    }
}
    
def format_row_as_string(rule_row,class_name,feature_names,feature_mapper):
    rule_pairs = {name:rule_row[name] for name in feature_names if not (rule_row[name]==None or rule_row[name]!=rule_row[name])}
    rule_strings = [f"{condition}={feature_mapper[condition][int(float(rule_pairs[condition]))]}" for condition in rule_pairs]
    long_rule_string = ", ".join(rule_strings)
    
    class_string = f"{class_name}={rule_row['class_val']}"
    
    odds_ratio_string = f"OR={round(rule_row['odds_ratio'],2)}"
    odds_ratio_string_with_ci = f"{odds_ratio_string} ({round(rule_row['OR_95CI_lower'],2)}-{round(rule_row['OR_95CI_upper'],2)})"
    raw_number_string = f"n={rule_row['support_count']}/{rule_row['coverage_count']}"
    coverage_string = f"Coverage={round(100*rule_row['coverage'],2)}%"
    posterior_string = f"Confidence={round(100*rule_row['confidence/posterior'],2)}%"
    support_string = f"Support={round(100*rule_row['support'],2)}%"
    p_value_string = f"p-value={round(rule_row['p_value'],4)}"
    lr_string = f"LR+={round(rule_row['LR+'],2)} "
    lr_string_with_ci = f"{lr_string} ({round(rule_row['LR+_95CI_lower'],2)}-{round(rule_row['LR+_95CI_upper'],2)})"
    
    stat_string = f"{raw_number_string};{odds_ratio_string_with_ci}; {lr_string_with_ci}; {support_string}; {coverage_string}; {posterior_string}; {p_value_string}"
    
    long_string = f"{long_rule_string} ==> {class_string} ({stat_string})"
    return long_string

def get_df_text_list(df,class_name,feature_names,feature_mapper):
    to_return = []
    for i in tqdm(range(len(df))):
        to_return.append(format_row_as_string(df.iloc[i,],class_name,feature_names,feature_mapper))
    return to_return


## Runs the method to obtain text list

In [184]:
text_list = get_df_text_list(big_rules_df,'GDM',all_attributes,mapper_dict)

100%|██████████| 132649/132649 [00:49<00:00, 2654.71it/s]


### Example of an interpretation

In [185]:
text_list[0]

'Diabetes_History_Cat=No Family DM History, High_BP_Cat=Has High Blood Pressure, Gravidity_Cat=Was Pregnant Previously, Smoking_Cat=No Tobacco use within 3 months, Drinking_Cat=Consumed Alcohol within 3 months, Poverty_Cat=Above federal poverty level ==> GDM=1 (n=5/5;OR=inf (nan-nan); LR+=inf  (nan-nan); Support=0.05%; Coverage=0.05%; Confidence=100.0%; p-value=0.0)'

### insert interpretation at the beginning of the dataframe for visibility

In [187]:
big_rules_df.insert(1,"Intepretation",text_list)

In [188]:
big_rules_df

,rule_id,Intepretation,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,...,coverage,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper
0,655705,"Diabetes_History_Cat=No Family DM History, Hig...",6,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
1,688458,"Diabetes_History_Cat=No Family DM History, PCO...",7,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
2,688459,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
3,688460,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
4,726752,"Diabetes_History_Cat=No Family DM History, PCO...",8,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132644,1606103,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,NaN,1.0,1.0,1.0,...,0.072233,0.000546,0.007564,0.166800,0.068773,0.404552,1.293965e-07,0.177879,0.074242,0.426190
132645,1606104,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",8,2.0,2.0,1.0,NaN,NaN,1.0,NaN,...,0.072342,0.000546,0.007553,0.166525,0.068660,0.403885,1.292404e-07,0.177609,0.074129,0.425540
132646,1606105,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",7,2.0,2.0,1.0,NaN,NaN,1.0,NaN,...,0.072451,0.000546,0.007541,0.166252,0.068547,0.403219,1.293303e-07,0.177339,0.074017,0.424891
132647,1606106,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",7,2.0,2.0,1.0,NaN,NaN,1.0,1.0,...,0.073981,0.000546,0.007386,0.162507,0.067008,0.394111,9.095415e-08,0.173644,0.072480,0.416010


# Remove rules which has "missing data" as part of the clauses


In [203]:
rules_with_zero = (big_rules_df[all_attributes] == 0).any(axis=1)
big_rules_df = big_rules_df[~rules_with_zero].copy()
big_rules_df

,rule_id,Intepretation,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,...,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper,q_value
0,655705,"Diabetes_History_Cat=No Family DM History, Hig...",6,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
1,688458,"Diabetes_History_Cat=No Family DM History, PCO...",7,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
2,688459,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
3,688460,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
4,726752,"Diabetes_History_Cat=No Family DM History, PCO...",8,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132494,1605953,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,NaN,1.0,1.0,...,0.000765,0.009174,0.201220,0.094907,0.426621,9.631310e-08,0.216090,0.103407,0.451566,0.000004
132495,1605954,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",10,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.000765,0.009174,0.201220,0.094907,0.426621,9.631310e-08,0.216090,0.103407,0.451566,0.000004
132496,1605955,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.000765,0.009150,0.200639,0.094634,0.425385,9.756253e-08,0.215520,0.103135,0.450371,0.000004
132507,1605966,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,NaN,1.0,1.0,...,0.000765,0.009067,0.198629,0.093689,0.421112,6.652442e-08,0.213548,0.102194,0.446236,0.000003


# After filtering, add in q-value for multiple hypothesis testing correction

Insert addition q-value column (Benjamin-Hochberg) FDR correction at alpha =0.05

In [204]:
from statsmodels.stats.multitest import fdrcorrection
q_values = fdrcorrection(big_rules_df['p_value'].to_numpy(),alpha=0.05,
                         method='indep',is_sorted=False)
_,big_rules_df['q_value'] = q_values
big_rules_df

,rule_id,Intepretation,num_clauses,AgeCat,BMI_Cat,Race,Diabetes_History_Cat,PCOS_Cat,High_BP_Cat,Gravidity_Cat,...,support,confidence/posterior,odds_ratio,OR_95CI_lower,OR_95CI_upper,p_value,LR+,LR+_95CI_lower,LR+_95CI_upper,q_value
0,655705,"Diabetes_History_Cat=No Family DM History, Hig...",6,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
1,688458,"Diabetes_History_Cat=No Family DM History, PCO...",7,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
2,688459,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
3,688460,"Diabetes_History_Cat=No Family DM History, Hig...",7,NaN,NaN,NaN,1.0,NaN,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
4,726752,"Diabetes_History_Cat=No Family DM History, PCO...",8,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,0.000546,1.000000,inf,NaN,NaN,1.141496e-07,inf,NaN,NaN,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132494,1605953,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,NaN,1.0,1.0,...,0.000765,0.009174,0.201220,0.094907,0.426621,9.631310e-08,0.216090,0.103407,0.451566,0.000004
132495,1605954,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",10,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.000765,0.009174,0.201220,0.094907,0.426621,9.631310e-08,0.216090,0.103407,0.451566,0.000004
132496,1605955,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.000765,0.009150,0.200639,0.094634,0.425385,9.756253e-08,0.215520,0.103135,0.450371,0.000004
132507,1605966,"AgeCat=[18-25], BMI_Cat=[18.5-25), Race=Non-Hi...",9,2.0,2.0,1.0,1.0,NaN,1.0,1.0,...,0.000765,0.009067,0.198629,0.093689,0.421112,6.652442e-08,0.213548,0.102194,0.446236,0.000003


# Write the dataframe 

In [206]:
big_rules_df.to_csv("data_source/GDM_Association_Rules_2021_10_13_filtered.csv",index=False)